# Objective

Finding the best rental listings in Calgary.

Criteria:

(1): Affordable price (<= 2000 CAD)
(2): Within the catchment areas of good elementary schools (school rating >= 7)
(3): Low crime rate (below 50th percentile of historic crime count)

In [6]:
import sqlite3
import pandas as pd
import numpy as np
import re

### Collecting The Data

In [40]:
conn = sqlite3.connect('database.db')

df_report = pd.read_sql_query('''
                              SELECT
    rl.community,
    rl."type",
    rl.beds,
    rl.sq_feet, 
    rl.price,
    rl.baths,
    rl.cats,
    rl.dogs,
    rl.utilities_included,
    rl.latitude,
    rl.longitude,
    c.crime_count,
    ROUND(c.crime_pct, 2) AS crime_percentile,
    (SELECT s.name FROM schools s
        JOIN school_ranking sr ON sr.school_id = s.school_id
        WHERE sr.school_id = swwz.school_id
        ORDER BY sr.school_rating DESC
        LIMIT 1) AS highest_rated_school_name,
    MAX(sr.school_rating) AS highest_school_rating,    
    CASE 
        WHEN sl.school_id IS NOT NULL THEN 'Required'
        ELSE 'Not Required'
    END AS lottery_requirement
    
FROM
    rental_listings rl
    INNER JOIN schools_within_walk_zone swwz ON swwz.listing_id = rl.id
    INNER JOIN listing_with_crime lwc ON lwc.listing_id = rl.id
    INNER JOIN crime c ON c.id = lwc.crime_id
    INNER JOIN schools s ON s.school_id = swwz.school_id
    INNER JOIN school_ranking sr ON sr.school_id = s.school_id
    LEFT JOIN school_lottery sl ON sl.school_id = s.school_id 
GROUP BY
    rl.id
ORDER BY
    highest_school_rating DESC,
    price ASC
                              ''',conn)

conn.close()

### Data Transform and Load

#### Basic Analysis (Descriptive)

In [41]:
conn = sqlite3.connect('database.db')

df_listing = pd.read_sql_query('''
                            SELECT community,
                                  "type",
                                  beds,
                                  sq_feet, 
                                  price,
                                  baths,
                                  cats,
                                  dogs,
                                  utilities_included,
                                  latitude,
                                  longitude 
                            FROM
                            rental_listings
                              ''',conn)

conn.close()

In [42]:
df_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3718 entries, 0 to 3717
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   community           3718 non-null   object 
 1   type                3718 non-null   object 
 2   beds                3675 non-null   object 
 3   sq_feet             3675 non-null   object 
 4   price               3718 non-null   object 
 5   baths               3675 non-null   object 
 6   cats                3675 non-null   float64
 7   dogs                3675 non-null   float64
 8   utilities_included  3718 non-null   object 
 9   latitude            3718 non-null   float64
 10  longitude           3718 non-null   float64
dtypes: float64(4), object(7)
memory usage: 319.6+ KB


In [43]:
df_listing.head(3)

,community,type,beds,sq_feet,price,baths,cats,dogs,utilities_included,latitude,longitude
0,Eau Claire,Condo Unit,1,600,1900,1,0.0,0.0,"[""Heat"", ""Electricity"", ""Water"", ""Cable"", ""Int...",51.050855,-114.077663
1,Hamptons,Townhouse,3,1700,2350,2.5,0.0,0.0,"[""See Full Description"", """"]",51.145227,-114.142047
2,Huntington Hills,Room For Rent,1,,750,1,0.0,0.0,"[""Heat"", ""Electricity"", ""Water"", ""Cable"", """"]",51.123184,-114.058484


apply value_counts(dropna=False) to examine the data in each column:

In [44]:
df_listing[['cats','dogs']].value_counts(dropna=False)

cats  dogs
0.0   0.0     2303
2.0   2.0     1059
      0.0      167
0.0   2.0      146
NaN   NaN       43
Name: count, dtype: int64

##### clean['baths'] column:

In [45]:
df_listing['baths'].value_counts()

baths
1       1793
2.5      723
2        624
1.5      186
3.5      177
3        120
4         26
4.5       19
5          5
None       1
7.5        1
Name: count, dtype: int64

In [46]:
# Replace "None" with NaN
df_listing['baths'] = df_listing['baths'].replace('None', np.nan)

# Convert the column to float
df_listing['baths'] = df_listing['baths'].astype(float)


In [47]:
df_listing['baths'].value_counts()

baths
1.0    1793
2.5     723
2.0     624
1.5     186
3.5     177
3.0     120
4.0      26
4.5      19
5.0       5
7.5       1
Name: count, dtype: int64

##### clean ['beds'] column:

it contains mixed data types. I need to convert all string before string manipulation, then converted back to int. The '+den" information will be stored in a new column.

In [48]:
df_listing['beds'].value_counts()

beds
2         1130
1          844
3          763
4          238
3+den      230
2+den      151
1+den      121
4+den       75
studio      46
5           46
5+den       20
6           10
7+den        1
Name: count, dtype: int64

In [49]:
def clean_beds(df):
    """
    Cleans the 'beds' column in a DataFrame by converting it to integers and adding a 'has_den' boolean column.
    
    This function updates the 'beds' column by converting all entries to integers, with 'studio' and 'None' being treated as 0.
    A new boolean column 'has_den' is added to indicate whether the original entry included a '+den'.
    
    Parameters:
    df (pd.DataFrame): The DataFrame with the 'beds' column to clean.
    
    Returns:
    pd.DataFrame: The DataFrame with the cleaned 'beds' column and the new 'has_den' column.
    """
    # Create 'has_den' column based on the presence of '+den' in the 'beds' column
    df['has_den'] = df['beds'].astype(str).str.contains('+den', regex=False)
    
    # Convert all 'beds' values to strings, then replace '+den' with '' and 'studio' with '0'
    df['beds'] = df['beds'].astype(str).replace({r'\+den': '', 'studio': '0', 'None': '0'}, regex=True)
    
    # Convert the 'beds' column to integer
    df['beds'] = pd.to_numeric(df['beds'], errors='coerce', downcast='integer')
    
    return df

In [50]:
clean_beds(df_listing).beds.value_counts()

beds
2    1281
3     993
1     965
4     313
0      89
5      66
6      10
7       1
Name: count, dtype: int64

##### clean 'sq_feet' column

In [51]:
df_listing['sq_feet'].apply(type).value_counts()

sq_feet
<class 'int'>         2686
<class 'str'>          973
<class 'NoneType'>      43
<class 'float'>         16
Name: count, dtype: int64

In [52]:
# Function to clean and extract square footage from various data types
def clean_sq_feet(value):
    """
    Cleans the square feet data by handling integers, floats, NoneType, and strings differently.
    NaN is set for NoneType and empty strings. Floats are rounded to the nearest integer.
    Strings are processed using extract_square_feet function to extract numeric value.
    
    Parameters:
    value: The data value to be cleaned.
    
    Returns:
    int or NaN: The cleaned square feet as an integer or NaN.
    """
    # If the value is None (NoneType), return NaN
    if value is None:
        return np.nan
    
    # If the value is an integer, return it as it is
    elif isinstance(value, int):
        return value
    
    # If the value is a float, round it to the nearest integer
    elif isinstance(value, float):
        return round(value)
    
    # If the value is a string, use the extract_square_feet function
    elif isinstance(value, str):
        # If the string is empty, return NaN
        if value.strip() == '':
            return np.nan
        else:
            return extract_square_feet(value)
    
    # If the value is of any other type, return NaN
    else:
        return np.nan

# Function to clean and extract square footage from strings
def extract_square_feet(value):
    # Regular expression to find the first number in the string with VERBOSE mode for commenting
    pattern = re.compile(r"""
        \d+              # Match one or more digits
        \s*              # Match any whitespace characters (zero or more)
        (?:              # Non-capturing group for the following:
            sq           # Match 'sq'
            (?:uare)?    # Optionally match 'uare' for 'square'
            \.?          # Optionally match a literal '.' for abbreviation
            \s*ft        # Match ' ft' with optional space before 'ft'
            \.?          # Optionally match a literal '.' for abbreviation
        |                # OR
            sf           # Match 'sf' for square feet
        |                # OR
            ft²          # Match 'ft²' for square feet in square notation
        |                # OR
            sqft         # Match 'sqft' for square feet
        )?               # Make the entire group optional
        """, re.VERBOSE)

    # Remove commas for consistency
    value = value.lower().replace(',', '')

    # Search for the pattern in the cleaned string
    match = pattern.search(value)
    
    if match:
        # Extract the number and remove any non-numeric characters
        square_feet = re.sub(r'[^\d]', '', match.group())
        try:
            # Convert to integer
            return int(square_feet)
        except ValueError:
            # If conversion fails, return NaN
            return np.nan
    else:
        # If no number found or if it's a complex description, return NaN
        return np.nan


In [53]:
df_listing['sq_feet'] = df_listing['sq_feet'].apply(clean_sq_feet).astype('Int64')

In [54]:
df_listing

,community,type,beds,sq_feet,price,baths,cats,dogs,utilities_included,latitude,longitude,has_den
0,Eau Claire,Condo Unit,1,600,1900,1.0,0.0,0.0,"[""Heat"", ""Electricity"", ""Water"", ""Cable"", ""Int...",51.050855,-114.077663,False
1,Hamptons,Townhouse,3,1700,2350,2.5,0.0,0.0,"[""See Full Description"", """"]",51.145227,-114.142047,False
2,Huntington Hills,Room For Rent,1,<NA>,750,1.0,0.0,0.0,"[""Heat"", ""Electricity"", ""Water"", ""Cable"", """"]",51.123184,-114.058484,False
3,Evergreen,Townhouse,3,1200,2050,1.5,2.0,0.0,"[""""]",50.912098,-114.105428,False
4,Scarboro,Basement,0,600,1200,1.0,0.0,0.0,"[""Electricity"", ""Water"", ""Cable"", "" "", ""Heat"",...",51.040318,-114.112280,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3713,Copperfield,House,4,1866,2950,3.5,0.0,0.0,"[""""]",50.909535,-113.936320,False
3714,Highland Park,Main Floor,2,1000,1700,1.0,0.0,0.0,"[""""]",51.083453,-114.055931,True
3715,Mission,Condo Unit,1,809,2350,1.0,0.0,0.0,"[""Heat"", ""Cable"", ""Electricity"", ""Internet"", ""...",51.037200,-114.064131,False
3716,Livingston,Townhouse,3,1300,2300,1.5,2.0,2.0,"[""""]",51.186828,-114.049578,False


##### Clean ['cat'] and ['dogs'] columns. 

They are either 2 or 0 or Null.  So make them as boolean columns instead. 

In [55]:
df_listing[['cats','dogs']].value_counts(dropna=False)

cats  dogs
0.0   0.0     2303
2.0   2.0     1059
      0.0      167
0.0   2.0      146
NaN   NaN       43
Name: count, dtype: int64

In [56]:

def clean_pet(value):
    # Check if the value is NaN or 0.0 and return False, otherwise return True
    return not (pd.isna(value) or value == 0.0)


In [57]:
df_listing['cats'] = df_listing['cats'].apply(clean_pet)
df_listing['dogs'] = df_listing['dogs'].apply(clean_pet)

In [58]:
df_listing.head()

,community,type,beds,sq_feet,price,baths,cats,dogs,utilities_included,latitude,longitude,has_den
0,Eau Claire,Condo Unit,1,600,1900,1.0,False,False,"[""Heat"", ""Electricity"", ""Water"", ""Cable"", ""Int...",51.050855,-114.077663,False
1,Hamptons,Townhouse,3,1700,2350,2.5,False,False,"[""See Full Description"", """"]",51.145227,-114.142047,False
2,Huntington Hills,Room For Rent,1,<NA>,750,1.0,False,False,"[""Heat"", ""Electricity"", ""Water"", ""Cable"", """"]",51.123184,-114.058484,False
3,Evergreen,Townhouse,3,1200,2050,1.5,True,False,"[""""]",50.912098,-114.105428,False
4,Scarboro,Basement,0,600,1200,1.0,False,False,"[""Electricity"", ""Water"", ""Cable"", "" "", ""Heat"",...",51.040318,-114.112280,False


#### Drop ['utilities_included'] column

In [59]:
df_listing['utilities_included'].value_counts(normalize=True)

utilities_included
[""]                                                                             0.386767
["Heat", "Water"]                                                                0.137708
["See Full Description"]                                                         0.090371
["Heat", "Electricity", "Water"]                                                 0.038999
["Heat", "Electricity", "Water", "Internet"]                                     0.035772
                                                                                   ...   
["Heat", "Cable", "Water", "Internet", "Electricity"]                            0.000269
["Water", "See Full Description", "Electricity", "Heat", "Internet"]             0.000269
["Heat", "Cable", "Electricity", "Water", "Internet", "See Full Description"]    0.000269
["Internet", "Water", "Electricity", "Heat", "Cable"]                            0.000269
["Heat", "Internet", "Water", "See Full Description"]                            

38.7% of data is empty string, 9% is "See Full Description". Looking at the rest of the data, many list also has an item called "See Full Description". Given such large proportion of missing data, I will simply drop the column. 

In [60]:
df_listing.drop(columns=['utilities_included'], axis = 1, inplace=True)

In [63]:
df_listing.head()

,community,type,beds,sq_feet,price,baths,cats,dogs,latitude,longitude,has_den
0,Eau Claire,Condo Unit,1,600,1900,1.0,False,False,51.050855,-114.077663,False
1,Hamptons,Townhouse,3,1700,2350,2.5,False,False,51.145227,-114.142047,False
2,Huntington Hills,Room For Rent,1,<NA>,750,1.0,False,False,51.123184,-114.058484,False
3,Evergreen,Townhouse,3,1200,2050,1.5,True,False,50.912098,-114.105428,False
4,Scarboro,Basement,0,600,1200,1.0,False,False,51.040318,-114.112280,False


#### Scatter Matrix

### Distribution of Property Type

bar chart

box-and-whisker plot


In [68]:
df_listing['type'].value_counts()

type
Condo Unit       613
House            609
Basement         591
Townhouse        487
Apartment        460
Main Floor       346
Room For Rent    337
Duplex           215
Storage           29
Parking Spot      14
Loft              12
Acreage            3
Mobile             1
Vacation Home      1
Name: count, dtype: int64

In [18]:

df = pd.DataFrame({'num_legs': [4, 2], 'num_wings': [1, np.nan]},
                  index=['dog', 'hawk'])

# Set data types for each column separately
df['num_legs'] = df['num_legs'].astype(int)
df['num_wings'] = df['num_wings'].astype('Int64')

for row in df.itertuples():
    print(row)

Pandas(Index='dog', num_legs=4, num_wings=1)
Pandas(Index='hawk', num_legs=2, num_wings=<NA>)


In [21]:
type(pd.NA)

pandas._libs.missing.NAType

In [26]:
set(df['num_legs'].unique())

{2, 4}

### Property Prices

- Price per area? in scatter plot
in subplots

- Price histogram

- Area in square meters histogram

### Utilities cost / Deposit / Property publisher (whether listed by agent or individuals) / Floor number? 

### Geo Visualization using Folium

- convert address to coordinate using GeoPy library.

### Rent Dynamics (how fast listings disappear?)

### Word Cloud of the listing titles